In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Pre-Processing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/M.S. Courses/DS 615 Data Driven Modeling/Final Project/Full Flight.csv', index_col = False)
df_copy = df.copy()
df
#What aircrafts at which airports during a specific time period
#specs on capacity of aircraft.
#Passenger information on each of these flights.
#Airport arrival and departures (potentially)
#departures only????

#Started thinking about how we can just find financial data using the models associated with Aircraft Type. Use that data to associate flights and miles between locations.

# Sample datasets over two days that we can start testing models on.


,#,Date,Airport,Flight\nType,User\nClass,Weight\nClass,Physical\nClass,Aircraft\nType,Business\nJet,Regional\nJet,...,Airplane\nApproach\nCategory,Airplane\nDesign\nGroup,Taxiway\nDesign\nGroup,Departures,Arrivals,Total\nOperations,Departure\nSeats,Average\nDeparture\nSeats,Arrival\nSeats,Average\nArrival\nSeats
0,1,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A332 - Airbus A330-200,No,No,...,C,V,5,0,2,2,0,0.0,506,253.0
1,2,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A333 - Airbus A330-300,No,No,...,C,V,5,0,9,9,0,0.0,"2,975",330.0
2,3,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A359 - Airbus 350-900,No,Unknown,...,C,V,5,0,6,6,0,0.0,"1,884",314.0
3,4,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A35K - Airbus A350-1000,No,Unknown,...,D,VI,6,0,1,1,0,0.0,369,369.0
4,5,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,B744 - Boeing 747-400,No,No,...,D,V,5,0,1,1,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14819,14820,10/01/2022,TPA - Tampa,Domestic,Air Taxi,Small Eqpt,Jet,E55P - Embraer Phenom 300,Yes,No,...,B,II,1B,1,2,3,8,8.0,16,8.0
14820,Total:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"68,633","68,527","137,160","8,273,355",120.0,"8,268,501",120.0
14821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14822,Report created on Mon Oct 28 23:56:25 EDT 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.drop(columns=['#'])
df = df.iloc[:-3]
df

,Date,Airport,Flight\nType,User\nClass,Weight\nClass,Physical\nClass,Aircraft\nType,Business\nJet,Regional\nJet,Business\nAviation,Airplane\nApproach\nCategory,Airplane\nDesign\nGroup,Taxiway\nDesign\nGroup,Departures,Arrivals,Total\nOperations,Departure\nSeats,Average\nDeparture\nSeats,Arrival\nSeats,Average\nArrival\nSeats
0,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A332 - Airbus A330-200,No,No,No,C,V,5,0,2,2,0,0.0,506,253.0
1,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A333 - Airbus A330-300,No,No,No,C,V,5,0,9,9,0,0.0,"2,975",330.0
2,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A359 - Airbus 350-900,No,Unknown,No,C,V,5,0,6,6,0,0.0,"1,884",314.0
3,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A35K - Airbus A350-1000,No,Unknown,No,D,VI,6,0,1,1,0,0.0,369,369.0
4,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,B744 - Boeing 747-400,No,No,No,D,V,5,0,1,1,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14816,10/01/2022,TPA - Tampa,Domestic,Air Taxi,Medium Commuter Eqpt,Jet,CL35 - Bombardier Challenger 300,Yes,Unknown,Yes,C,II,1B,1,2,3,8,8.0,16,8.0
14817,10/01/2022,TPA - Tampa,Domestic,Air Taxi,Medium Commuter Eqpt,Jet,GLEX - Bombardier BD-700 Global Express,Yes,No,Yes,C,III,2B,1,1,2,11,11.0,11,11.0
14818,10/01/2022,TPA - Tampa,Domestic,Air Taxi,Medium Commuter Eqpt,Jet,GLF6 - Gulfstream,Yes,No,Yes,C,III,2B,0,1,1,0,0.0,15,15.0
14819,10/01/2022,TPA - Tampa,Domestic,Air Taxi,Small Eqpt,Jet,E55P - Embraer Phenom 300,Yes,No,Yes,B,II,1B,1,2,3,8,8.0,16,8.0


In [ ]:
df_type = df['Aircraft\nType'].unique()
df_type = pd.Series(df_type)
df_type

,0
0,A332 - Airbus A330-200
1,A333 - Airbus A330-300
2,A359 - Airbus 350-900
3,A35K - Airbus A350-1000
4,B744 - Boeing 747-400
...,...
268,A139 - Agusta AB-139
269,HELO - Helicoper
270,DV20 - Diamond DV-20
271,H47 - Boeing CH-47 Chinook


In [ ]:
commercial_aircraft = ['A332', 'A333', 'A359', 'A35K', 'A319', 'A320', 'A321', 'A20N',
                       'B744', 'B763', 'B764', 'B772', 'B77L', 'B77W', 'B752', 'B712',
                       'B738', 'B739', 'B789', 'B748', 'B753', 'B38M', 'B39M', 'B737']

Class = ['Air Carrier']

# Create the first mask for commercial aircraft types
aircraft_mask = df['Aircraft\nType'].astype(str).apply(lambda x: x.split(' - ')[0] in commercial_aircraft)

# Apply the first mask to get an intermediate DataFrame
df_commercial = df[aircraft_mask]

# Create the second mask for air carriers
carrier_mask = df_commercial['User\nClass'].astype(str).apply(lambda x: x.split(' - ')[0] in Class)

# Apply the second mask to the intermediate DataFrame
df_commercial = df_commercial[carrier_mask]


In [ ]:
aircraft_type = df_commercial['Aircraft\nType'].unique()
print(aircraft_type)
print(df_commercial.columns)
df_commercial

['A332 - Airbus A330-200' 'A333 - Airbus A330-300' 'A359 - Airbus 350-900'
 'A35K - Airbus A350-1000' 'B744 - Boeing 747-400' 'B763 - Boeing 767-300'
 'B764 - Boeing 767-400' 'B772 - Boeing 777-200'
 'B77L - Boeing 777-200LRF/LR' 'B77W - Boeing 777-300ER'
 'B752 - Boeing 757-200' 'A20N - Airbus A320 Neo' 'A319 - Airbus A319'
 'A320 - Airbus A320 All Series' 'A321 - Airbus A321 All Series'
 'B712 - Boeing 717-200' 'B738 - Boeing 737-800' 'B739 - Boeing 737-900'
 'B789 - Boeing 787-9 Dreamliner' 'B748 - Boeing 747-8'
 'B753 - Boeing 757-300' 'B38M - Boeing 737 Max 8'
 'B39M - Boeing 737 Max 9' 'B737 - Boeing 737-700']
Index(['Date', 'Airport', 'Flight\nType', 'User\nClass', 'Weight\nClass',
       'Physical\nClass', 'Aircraft\nType', 'Business\nJet', 'Regional\nJet',
       'Business\nAviation', 'Airplane\nApproach\nCategory',
       'Airplane\nDesign\nGroup', 'Taxiway\nDesign\nGroup', 'Departures',
       'Arrivals', 'Total\nOperations', 'Departure\nSeats',
       'Average\nDeparture\nS

,Date,Airport,Flight\nType,User\nClass,Weight\nClass,Physical\nClass,Aircraft\nType,Business\nJet,Regional\nJet,Business\nAviation,Airplane\nApproach\nCategory,Airplane\nDesign\nGroup,Taxiway\nDesign\nGroup,Departures,Arrivals,Total\nOperations,Departure\nSeats,Average\nDeparture\nSeats,Arrival\nSeats,Average\nArrival\nSeats
0,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A332 - Airbus A330-200,No,No,No,C,V,5,0,2,2,0,0.0,506,253.0
1,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A333 - Airbus A330-300,No,No,No,C,V,5,0,9,9,0,0.0,"2,975",330.0
2,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A359 - Airbus 350-900,No,Unknown,No,C,V,5,0,6,6,0,0.0,"1,884",314.0
3,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,A35K - Airbus A350-1000,No,Unknown,No,D,VI,6,0,1,1,0,0.0,369,369.0
4,09/21/2022,ATL - Atlanta,Foreign to US,Air Carrier,Heavy Eqpt,Jet,B744 - Boeing 747-400,No,No,No,D,V,5,0,1,1,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14765,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B39M - Boeing 737 Max 9,No,No,No,D,III,3,3,3,6,508,169.0,508,169.0
14766,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B712 - Boeing 717-200,No,No,No,C,III,2B,1,1,2,116,116.0,116,116.0
14767,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B737 - Boeing 737-700,No,No,No,C,III,3,48,51,99,"6,236",129.0,"6,626",129.0
14768,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B738 - Boeing 737-800,No,No,No,D,III,3,34,33,67,"5,448",160.0,"5,286",160.0


In [ ]:
df_commercial = df_commercial[df_commercial['Flight\nType'] == 'Domestic']
df_commercial

,Date,Airport,Flight\nType,User\nClass,Weight\nClass,Physical\nClass,Aircraft\nType,Business\nJet,Regional\nJet,Business\nAviation,Airplane\nApproach\nCategory,Airplane\nDesign\nGroup,Taxiway\nDesign\nGroup,Departures,Arrivals,Total\nOperations,Departure\nSeats,Average\nDeparture\nSeats,Arrival\nSeats,Average\nArrival\nSeats
42,09/21/2022,ATL - Atlanta,Domestic,Air Carrier,Heavy Eqpt,Jet,A332 - Airbus A330-200,No,No,No,C,V,5,0,1,1,0,0.0,253,253.0
43,09/21/2022,ATL - Atlanta,Domestic,Air Carrier,Heavy Eqpt,Jet,A333 - Airbus A330-300,No,No,No,C,V,5,1,0,1,335,335.0,0,0.0
44,09/21/2022,ATL - Atlanta,Domestic,Air Carrier,Heavy Eqpt,Jet,A359 - Airbus 350-900,No,Unknown,No,C,V,5,1,2,3,314,314.0,628,314.0
45,09/21/2022,ATL - Atlanta,Domestic,Air Carrier,Heavy Eqpt,Jet,B748 - Boeing 747-8,No,No,No,D,VI,6,2,3,5,728,364.0,"1,092",364.0
46,09/21/2022,ATL - Atlanta,Domestic,Air Carrier,Heavy Eqpt,Jet,B763 - Boeing 767-300,No,No,No,C,IV,5,10,10,20,"1,380",138.0,"1,380",138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14765,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B39M - Boeing 737 Max 9,No,No,No,D,III,3,3,3,6,508,169.0,508,169.0
14766,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B712 - Boeing 717-200,No,No,No,C,III,2B,1,1,2,116,116.0,116,116.0
14767,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B737 - Boeing 737-700,No,No,No,C,III,3,48,51,99,"6,236",129.0,"6,626",129.0
14768,10/01/2022,TPA - Tampa,Domestic,Air Carrier,Large Jet Eqpt,Jet,B738 - Boeing 737-800,No,No,No,D,III,3,34,33,67,"5,448",160.0,"5,286",160.0


In [ ]:
df = df_commercial[['Date','Airport','User\nClass','Aircraft\nType','Departures','Arrivals','Departure\nSeats','Arrival\nSeats']]
df.columns = ['Date', 'Airport', 'User Class', 'Aircraft Type', 'Departures', 'Arrivals', 'Departure Seats', 'Arrival Seats']
df = df.reset_index(drop=True)
df.index = np.arange(1, len(df) + 1)
df['Airport'] = df['Airport'].str[:3]
df

,Date,Airport,User Class,Aircraft Type,Departures,Arrivals,Departure Seats,Arrival Seats
1,09/21/2022,ATL,Air Carrier,A332 - Airbus A330-200,0,1,0,253
2,09/21/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0,335,0
3,09/21/2022,ATL,Air Carrier,A359 - Airbus 350-900,1,2,314,628
4,09/21/2022,ATL,Air Carrier,B748 - Boeing 747-8,2,3,728,"1,092"
5,09/21/2022,ATL,Air Carrier,B763 - Boeing 767-300,10,10,"1,380","1,380"
...,...,...,...,...,...,...,...,...
2028,10/01/2022,TPA,Air Carrier,B39M - Boeing 737 Max 9,3,3,508,508
2029,10/01/2022,TPA,Air Carrier,B712 - Boeing 717-200,1,1,116,116
2030,10/01/2022,TPA,Air Carrier,B737 - Boeing 737-700,48,51,"6,236","6,626"
2031,10/01/2022,TPA,Air Carrier,B738 - Boeing 737-800,34,33,"5,448","5,286"


In [ ]:
df['Airport'].unique()

array(['ATL', 'AUS', 'BNA', 'CHS', 'CLT', 'DAB', 'DFW', 'FLL', 'IAH',
       'MCO', 'MEM', 'MIA', 'MSY', 'MYR', 'RSW', 'SAT', 'SAV', 'TPA'],
      dtype=object)

In [ ]:
airports = ['ATL - Atlanta', 'AUS - Austin', 'BNA - Nashville',
            'CHS - Charleston', 'CLT - Charlotte', 'DAB - Daytona Beach',
            'DFW - Dallas/Ft. Worth', 'FLL - Fort Lauderdale', 'IAH - Houston',
            'MCO - Orlando', 'MEM - Memphis', 'MIA - Miami',
            'MSY - New Orleans', 'MYR - Myrtle Beach', 'RSW - Fort Myers',
            'SAT - San Antonio', 'SAV - Savannah', 'TPA - Tampa']

df = df.drop(columns=['Arrivals', 'Arrival Seats'])
df['Departures'] = pd.to_numeric(df['Departures'], errors='coerce')
df = df[df['Departures'] != 0]
df = df.reset_index(drop=True)
df.index = np.arange(1, len(df) + 1)
#df.to_csv('All_airport.csv')
airport_dfs = {}  # Dictionary to store the DataFrames

for airport in airports:
    airport_dfs[airport] = df[df['Airport'] == airport].copy()
    airport_dfs[airport].reset_index(drop=True, inplace=True)
    # optionally: airport_dfs[airport].name = airport  # gives the df a name

atl_df = airport_dfs['ATL - Atlanta']
aus_df = airport_dfs['AUS - Austin']
bna_df = airport_dfs['BNA - Nashville']
chs_df = airport_dfs['CHS - Charleston']
clt_df = airport_dfs['CLT - Charlotte']
dab_df = airport_dfs['DAB - Daytona Beach']
dfw_df = airport_dfs['DFW - Dallas/Ft. Worth']
fll_df = airport_dfs['FLL - Fort Lauderdale']
iah_df = airport_dfs['IAH - Houston']
mco_df = airport_dfs['MCO - Orlando']
mem_df = airport_dfs['MEM - Memphis']
mia_df = airport_dfs['MIA - Miami']
msy_df = airport_dfs['MSY - New Orleans']
myr_df = airport_dfs['MYR - Myrtle Beach']
rsw_df = airport_dfs['RSW - Fort Myers']
sat_df = airport_dfs['SAT - San Antonio']
sav_df = airport_dfs['SAV - Savannah']
tpa_df = airport_dfs['TPA - Tampa']

In [ ]:
clt_df

,Date,Airport,User Class,Aircraft Type,Departures,Departure Seats


In [ ]:
#grab financial data associate with flying each of these aircraft types including fuel costs in 2022
# metrics on each of the aircraft types
df

,Date,Airport,User Class,Aircraft Type,Departures,Departure Seats
1,09/21/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,335
2,09/21/2022,ATL,Air Carrier,A359 - Airbus 350-900,1,314
3,09/21/2022,ATL,Air Carrier,B748 - Boeing 747-8,2,728
4,09/21/2022,ATL,Air Carrier,B763 - Boeing 767-300,10,"1,380"
5,09/21/2022,ATL,Air Carrier,B764 - Boeing 767-400,3,735
...,...,...,...,...,...,...
1969,10/01/2022,TPA,Air Carrier,B39M - Boeing 737 Max 9,3,508
1970,10/01/2022,TPA,Air Carrier,B712 - Boeing 717-200,1,116
1971,10/01/2022,TPA,Air Carrier,B737 - Boeing 737-700,48,"6,236"
1972,10/01/2022,TPA,Air Carrier,B738 - Boeing 737-800,34,"5,448"


In [ ]:
Check = df[df['Aircraft Type'] == 'A319 - Airbus A319']
Check

#Thoughts about potential questions: How about crew capacity? These flights were departures that were already made since we are working with historical data the flights had enough staff to crew these flights.
#Can we and should we set an arbitrary demand regarding the number of people who are wanting to evacuate that way the model has a number of people to work towards referencing the resources needed?!
#Demand oriented based on cone of hurricane #########

# Summative pool on each aircraft typer per day at each airport

,Date,Airport,User Class,Aircraft Type,Departures,Departure Seats
11,09/21/2022,ATL,Air Carrier,A319 - Airbus A319,30,"3,834"
22,09/21/2022,AUS,Air Carrier,A319 - Airbus A319,29,"3,632"
33,09/21/2022,BNA,Air Carrier,A319 - Airbus A319,12,"1,518"
43,09/21/2022,CHS,Air Carrier,A319 - Airbus A319,8,"1,004"
53,09/21/2022,CLT,Air Carrier,A319 - Airbus A319,43,"5,344"
...,...,...,...,...,...,...
1931,10/01/2022,MSY,Air Carrier,A319 - Airbus A319,16,"2,038"
1940,10/01/2022,MYR,Air Carrier,A319 - Airbus A319,6,768
1948,10/01/2022,SAT,Air Carrier,A319 - Airbus A319,11,"1,388"
1956,10/01/2022,SAV,Air Carrier,A319 - Airbus A319,4,502


In [ ]:
x = 3632/29
x

125.24137931034483

In [ ]:
Check = df[(df['Aircraft Type'] == 'B763 - Boeing 767-300') & (df['Departures'] == 1)]
Check
# How we could split the data for departures with an aircraft on the same day to show that there are multiple going out. This could increase the size of our records in our DF.

,Date,Airport,User Class,Aircraft Type,Departures,Departure Seats
50,09/21/2022,CLT,Air Carrier,B763 - Boeing 767-300,1,0
99,09/21/2022,MCO,Air Carrier,B763 - Boeing 767-300,1,230
254,09/22/2022,DFW,Air Carrier,B763 - Boeing 767-300,1,0
280,09/22/2022,IAH,Air Carrier,B763 - Boeing 767-300,1,230
431,09/23/2022,CLT,Air Carrier,B763 - Boeing 767-300,1,0
444,09/23/2022,DFW,Air Carrier,B763 - Boeing 767-300,1,0
622,09/24/2022,CLT,Air Carrier,B763 - Boeing 767-300,1,0
633,09/24/2022,DFW,Air Carrier,B763 - Boeing 767-300,1,0
807,09/25/2022,CLT,Air Carrier,B763 - Boeing 767-300,1,0
831,09/25/2022,FLL,Air Carrier,B763 - Boeing 767-300,1,230


In [ ]:
print(df['Airport'].nunique())
df['Airport'].unique()

18


array(['ATL', 'AUS', 'BNA', 'CHS', 'CLT', 'DAB', 'DFW', 'FLL', 'IAH',
       'MCO', 'MEM', 'MIA', 'MSY', 'MYR', 'RSW', 'SAT', 'SAV', 'TPA'],
      dtype=object)

In [ ]:
from geopy.distance import great_circle

# Airport codes with their latitudes and longitudes
airport_coords = {
    'ATL': (33.6407, -84.4277),   # Atlanta
    'AUS': (30.1945, -97.6699),   # Austin
    'BNA': (36.1245, -86.6782),   # Nashville
    'CHS': (32.8986, -80.0405),   # Charleston
    'CLT': (35.214, -80.9431),    # Charlotte
    'DAB': (29.1799, -81.0581),   # Daytona Beach
    'DFW': (32.8998, -97.0403),   # Dallas/Ft. Worth
    'FLL': (26.0726, -80.1527),   # Fort Lauderdale
    'IAH': (29.9902, -95.3368),   # Houston
    'MCO': (28.4312, -81.3081),   # Orlando
    'MEM': (35.0423, -89.9767),   # Memphis
    'MIA': (25.7959, -80.2870),   # Miami
    'MSY': (29.9934, -90.2580),   # New Orleans
    'MYR': (33.6795, -78.9283),   # Myrtle Beach
    'RSW': (26.5362, -81.7552),   # Fort Myers
    'SAT': (29.5337, -98.4698),   # San Antonio
    'SAV': (32.1276, -81.2022),   # Savannah
    'TPA': (27.9755, -82.5332)    # Tampa
}

# Reference airports: MCO and DAB
ref_airports = {'MCO': airport_coords['MCO'], 'DAB': airport_coords['DAB']}

# Calculate distances
distances = {airport: {ref: great_circle(coords, ref_coords).miles
                       for ref, ref_coords in ref_airports.items()}
             for airport, coords in airport_coords.items()}

distance_df = pd.DataFrame(distances).T
distance_df.index.name = 'Airport'
distance_df.columns = ['Distance to MCO (Orlando)', 'Distance to DAB (Daytona Beach)']

# Display the tidy DataFrame
distance_df.reset_index()

,Airport,Distance to MCO (Orlando),Distance to DAB (Daytona Beach)
0,ATL,404.495411,366.647959
1,AUS,992.367447,998.688582
2,BNA,616.940079,580.351585
3,CHS,317.719054,263.900395
4,CLT,469.135263,416.970723
5,DAB,53.899062,0.000000
6,DFW,983.452333,979.305819
7,FLL,177.742903,221.729716
8,IAH,852.305052,859.208015
9,MCO,0.000000,53.899062


In [ ]:
def find_first_non_zero_seats(group):
    """Finds the first non-zero 'Departure Seats' value in a group."""
    non_zero_seats = group[group != 0]
    if not non_zero_seats.empty:
        return group.loc[non_zero_seats.idxmax()]  # Use idxmax
    else:
        return None

# Create Flight_Cost DataFrame without setting index
Flight_Cost = pd.DataFrame({'Aircraft Type': df['Aircraft Type'].unique()})
Flight_Cost['Fuel Cost(Avg)'] = 3.49
Flight_Cost['Passenger Capacity'] = None  # Initialize Passenger Capacity column

# Filter, group, and apply the optimized function
passenger_capacity = df[df['Departures'] == 1].groupby('Aircraft Type')['Departure Seats'].apply(find_first_non_zero_seats).reset_index(name='Passenger Capacity')

# Merge passenger capacity with Flight_Cost, ensuring correct column names
Flight_Cost = pd.merge(Flight_Cost, passenger_capacity, on='Aircraft Type', how='left', suffixes=('_existing', ''))  # Keep the original name for the right DataFrame
Flight_Cost.drop(columns = ['Passenger Capacity_existing'], inplace=True)
row_index = Flight_Cost[Flight_Cost['Aircraft Type'] == 'A319 - Airbus A319'].index[0]
#If you provide a ratio to the departures for the A319 = 1 then you get 125 departure seats available
Flight_Cost.loc[row_index, 'Passenger Capacity'] = 125
Flight_Cost['Passenger Capacity'] = Flight_Cost['Passenger Capacity'].fillna(0)

Flight_Cost

,Aircraft Type,Fuel Cost(Avg),Passenger Capacity
0,A333 - Airbus A330-300,3.49,335
1,A359 - Airbus 350-900,3.49,314
2,B748 - Boeing 747-8,3.49,364
3,B763 - Boeing 767-300,3.49,230
4,B764 - Boeing 767-400,3.49,245
5,B77L - Boeing 777-200LRF/LR,3.49,270
6,B77W - Boeing 777-300ER,3.49,224
7,B752 - Boeing 757-200,3.49,200
8,B753 - Boeing 757-300,3.49,240
9,A20N - Airbus A320 Neo,3.49,165


In [ ]:
distance_miles = distances['AUS']['DAB']  # Distance from AUS to DAB in miles
miles_to_nautical_miles = 1 / 1.15078  # Conversion factor from miles to nautical miles
fuel_efficiency_nautical_miles_per_gallon = 0.12  # Nautical miles per gallon for B748

# Convert distance to nautical miles
distance_nautical_miles = distance_miles * miles_to_nautical_miles

# Calculate fuel burned in gallons
fuel_burned_gallons = distance_nautical_miles / fuel_efficiency_nautical_miles_per_gallon
fuel_burned_gallons

7231.968623827905

In [ ]:
def calculate_fuel_burn(distance_miles, fuel_efficiency_nm_per_gallon, miles_to_nm_conversion=1 / 1.15078):
    """
    Calculates fuel burned for a given distance in miles based on fuel efficiency in nautical miles per gallon.

    Parameters:
        distance_miles (float): Distance in miles.
        fuel_efficiency_nm_per_gallon (float): Fuel efficiency in nautical miles per gallon.
        miles_to_nm_conversion (float): Conversion factor from miles to nautical miles.

    Returns:
        float: Fuel burned in gallons.
    """
    # Convert distance to nautical miles
    distance_nautical_miles = distance_miles * miles_to_nm_conversion
    # Calculate and return fuel burned in gallons
    return distance_nautical_miles / fuel_efficiency_nm_per_gallon

# Use the function with the same data
fuel_burned_AUS_DAB = calculate_fuel_burn(distance_miles, fuel_efficiency_nautical_miles_per_gallon)
print('Gallons of Fuel Burned:',fuel_burned_AUS_DAB)

Gallons of Fuel Burned: 7231.968623827905


In [ ]:
#Trying to find financial data including fuel costs and metrics for aircrafts
#Fuel per flight consumed based on
#Miles between airport locations.
#Fuel cost per flight based on location
Flight_Cost['Aircraft Type'].unique()

array(['A333 - Airbus A330-300', 'A359 - Airbus 350-900',
       'B748 - Boeing 747-8', 'B763 - Boeing 767-300',
       'B764 - Boeing 767-400', 'B77L - Boeing 777-200LRF/LR',
       'B77W - Boeing 777-300ER', 'B752 - Boeing 757-200',
       'B753 - Boeing 757-300', 'A20N - Airbus A320 Neo',
       'A319 - Airbus A319', 'A320 - Airbus A320 All Series',
       'A321 - Airbus A321 All Series', 'B38M - Boeing 737 Max 8',
       'B39M - Boeing 737 Max 9', 'B712 - Boeing 717-200',
       'B737 - Boeing 737-700', 'B738 - Boeing 737-800',
       'B739 - Boeing 737-900', 'B744 - Boeing 747-400',
       'B772 - Boeing 777-200', 'B789 - Boeing 787-9 Dreamliner',
       'A332 - Airbus A330-200'], dtype=object)

In [ ]:
# Data for aircraft types and their corresponding nautical miles per gallon (nautical MPG)
data = {
    'Aircraft Type': [
        'A333 - Airbus A330-300', 'A359 - Airbus 350-900', 'B748 - Boeing 747-8',
        'B763 - Boeing 767-300', 'B764 - Boeing 767-400', 'B77L - Boeing 777-200LRF/LR',
        'B77W - Boeing 777-300ER', 'B752 - Boeing 757-200', 'B753 - Boeing 757-300',
        'A20N - Airbus A320 Neo', 'A319 - Airbus A319', 'A320 - Airbus A320 All Series',
        'A321 - Airbus A321 All Series', 'B38M - Boeing 737 Max 8', 'B39M - Boeing 737 Max 9',
        'B712 - Boeing 717-200', 'B737 - Boeing 737-700', 'B738 - Boeing 737-800',
        'B739 - Boeing 737-900', 'B744 - Boeing 747-400', 'B772 - Boeing 777-200',
        'B789 - Boeing 787-9 Dreamliner', 'A332 - Airbus A330-200'
    ],
    'Nautical MPG': [
        0.13, 0.14, 0.12, 0.13, 0.13, 0.12, 0.12, 0.13, 0.12, 0.15, 0.16, 0.14,
        0.13, 0.17, 0.16, 0.15, 0.15, 0.14, 0.14, 0.11, 0.12, 0.15, 0.13
    ]
}

Nautical_MPG = pd.DataFrame(data)

#not accurate MPG's for these aircraft at the moment. Need to find a resource that will provide that information.
Nautical_MPG

,Aircraft Type,Nautical MPG
0,A333 - Airbus A330-300,0.13
1,A359 - Airbus 350-900,0.14
2,B748 - Boeing 747-8,0.12
3,B763 - Boeing 767-300,0.13
4,B764 - Boeing 767-400,0.13
5,B77L - Boeing 777-200LRF/LR,0.12
6,B77W - Boeing 777-300ER,0.12
7,B752 - Boeing 757-200,0.13
8,B753 - Boeing 757-300,0.12
9,A20N - Airbus A320 Neo,0.15


In [ ]:
df_merge = pd.merge(df, Nautical_MPG, on='Aircraft Type', how='right')
df_merge = pd.merge(df_merge, Flight_Cost, on='Aircraft Type', how='left')
df = df_merge.drop(columns=['Departure Seats'])
df = pd.merge(df, distance_df, on='Airport', how='left')
df

,Date,Airport,User Class,Aircraft Type,Departures,Nautical MPG,Fuel Cost(Avg),Passenger Capacity,Distance to MCO (Orlando),Distance to DAB (Daytona Beach)
0,09/21/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,404.495411,366.647959
1,09/22/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,404.495411,366.647959
2,09/28/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,404.495411,366.647959
3,10/01/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,404.495411,366.647959
4,09/21/2022,ATL,Air Carrier,A359 - Airbus 350-900,1,0.14,3.49,314,404.495411,366.647959
...,...,...,...,...,...,...,...,...,...,...
1968,09/23/2022,ATL,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,404.495411,366.647959
1969,09/24/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,992.367447,998.688582
1970,09/27/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,992.367447,998.688582
1971,09/29/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,992.367447,998.688582


In [ ]:
# Function to calculate the estimated fuel cost
# Must convert miles to nautical miles for accuracy

df['NM Distance to MCO (Orlando)'] = df['Distance to MCO (Orlando)'] * 0.868976
df['NM Distance to DAB (Daytona Beach)'] = df['Distance to DAB (Daytona Beach)'] * 0.868976
df = df.drop(columns=['Distance to MCO (Orlando)', 'Distance to DAB (Daytona Beach)'])
df
def calculate_fuel_cost_MCO(row):
    # Total gallons needed = Distance / Nautical MPG
    total_gallons = row['NM Distance to MCO (Orlando)'] / row['Nautical MPG']
    # Total cost = Total gallons * Cost per gallon
    # Use row['Fuel Cost(Avg)'] instead of df['Fuel Cost(Avg)'] to access the value for the current row
    cost = total_gallons * row['Fuel Cost(Avg)']
    # Round the cost to the nearest tenth
    return round(cost, 2)

def calculate_fuel_cost_DB(row):
    # Total gallons needed = Distance / Nautical MPG
    total_gallons = row['NM Distance to DAB (Daytona Beach)'] / row['Nautical MPG']
    # Total cost = Total gallons * Cost per gallon
    # Use row['Fuel Cost(Avg)'] instead of df['Fuel Cost(Avg)'] to access the value for the current row
    cost = total_gallons * row['Fuel Cost(Avg)']
    # Round the cost to the nearest tenth
    return round(cost, 2)

# Add the new column "Estimated Fuel Cost"
df['Estimated Fuel Cost to MCO'] = df.apply(calculate_fuel_cost_MCO, axis=1)
df['Estimated Fuel Cost to DAB'] = df.apply(calculate_fuel_cost_DB, axis=1)
df

,Date,Airport,User Class,Aircraft Type,Departures,Nautical MPG,Fuel Cost(Avg),Passenger Capacity,NM Distance to MCO (Orlando),NM Distance to DAB (Daytona Beach),Estimated Fuel Cost to MCO,Estimated Fuel Cost to DAB
0,09/21/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
1,09/22/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
2,09/28/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
3,10/01/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
4,09/21/2022,ATL,Air Carrier,A359 - Airbus 350-900,1,0.14,3.49,314,351.496805,318.608277,8762.31,7942.45
...,...,...,...,...,...,...,...,...,...,...,...,...
1968,09/23/2022,ATL,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,351.496805,318.608277,9436.34,8553.41
1969,09/24/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,862.343494,867.836410,23150.61,23298.07
1970,09/27/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,862.343494,867.836410,23150.61,23298.07
1971,09/29/2022,AUS,Air Carrier,A332 - Airbus A330-200,1,0.13,3.49,253,862.343494,867.836410,23150.61,23298.07


In [ ]:
airports = ['ATL', 'AUS', 'BNA',
            'CHS', 'CLT', 'DAB',
            'DFW', 'FLL', 'IAH',
            'MCO', 'MEM', 'MIA',
            'MSY', 'MYR', 'RSW',
            'SAT', 'SAV', 'TPA']

df['Departures'] = pd.to_numeric(df['Departures'], errors='coerce')
df = df[df['Departures'] != 0]
df = df.reset_index(drop=True)
df.index = np.arange(1, len(df) + 1)
#df.to_csv('All_airport.csv')
airport_dfs = {}  # Dictionary to store the DataFrames

for airport in airports:
    airport_dfs[airport] = df[df['Airport'] == airport].copy()
    airport_dfs[airport].reset_index(drop=True, inplace=True)
    # optionally: airport_dfs[airport].name = airport  # gives the df a name

atl_df = airport_dfs['ATL']
aus_df = airport_dfs['AUS']
bna_df = airport_dfs['BNA']
chs_df = airport_dfs['CHS']
clt_df = airport_dfs['CLT']
dab_df = airport_dfs['DAB']
dfw_df = airport_dfs['DFW']
fll_df = airport_dfs['FLL']
iah_df = airport_dfs['IAH']
mco_df = airport_dfs['MCO']
mem_df = airport_dfs['MEM']
mia_df = airport_dfs['MIA']
msy_df = airport_dfs['MSY']
myr_df = airport_dfs['MYR']
rsw_df = airport_dfs['RSW']
sat_df = airport_dfs['SAT']
sav_df = airport_dfs['SAV']
tpa_df = airport_dfs['TPA']

In [ ]:
atl_df

,Date,Airport,User Class,Aircraft Type,Departures,Nautical MPG,Fuel Cost(Avg),Passenger Capacity,NM Distance to MCO (Orlando),NM Distance to DAB (Daytona Beach),Estimated Fuel Cost to MCO,Estimated Fuel Cost to DAB
0,09/21/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
1,09/22/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
2,09/28/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
3,10/01/2022,ATL,Air Carrier,A333 - Airbus A330-300,1,0.13,3.49,335,351.496805,318.608277,9436.34,8553.41
4,09/21/2022,ATL,Air Carrier,A359 - Airbus 350-900,1,0.14,3.49,314,351.496805,318.608277,8762.31,7942.45
...,...,...,...,...,...,...,...,...,...,...,...,...
188,09/28/2022,ATL,Air Carrier,B744 - Boeing 747-400,1,0.11,3.49,480,351.496805,318.608277,11152.03,10108.57
189,09/29/2022,ATL,Air Carrier,B744 - Boeing 747-400,2,0.11,3.49,480,351.496805,318.608277,11152.03,10108.57
190,09/30/2022,ATL,Air Carrier,B744 - Boeing 747-400,2,0.11,3.49,480,351.496805,318.608277,11152.03,10108.57
191,10/01/2022,ATL,Air Carrier,B744 - Boeing 747-400,1,0.11,3.49,480,351.496805,318.608277,11152.03,10108.57


In [ ]:
df.to_csv('Flights and Fuel Costs.csv')

In [32]:
df.Departures.unique()

array([  1,   2,   3,  10,   7,   4,   6,   9,   5,   8,  11, 112, 129,
       133,  12, 113, 128, 102,  82,  85, 108,  95,  14,  15,  17,  16,
        18,  31,  22,  25,  29,  27,  35,  20,  24,  40,  32,  26,  36,
        19,  33,  13,  21,  28,  30,  43,  57,  34,  51,  63,  38,  49,
        62,  42,  55,  37,  50,  61,  53,  41,  23,  59,  45,  60,  44,
        39,  58,  81,  83,  47,  68,  69,  56,  54,  80,  67, 147, 110,
       142, 155, 127, 167,  91, 151, 130, 161, 109, 146,  88, 149, 169,
       153, 160, 139, 107, 141,  84, 144,  89, 134, 148, 159, 124, 132,
       103, 123, 125, 106, 126, 120, 117,  77,  71,  78,  70,  66,  92,
        74,  65,  64,  52,  48,  90, 180,  79, 105, 207,  86, 205, 179,
       204,  46, 175,  72, 165, 191,  93, 200,  94, 101,  87,  97])

In [ ]:
#resource https://transtats.bts.gov/FUEL/

#References

https://www.cnn.com/2024/10/10/weather/map-charts-milton-hurricane-dg/index.html

https://www.nationsonline.org/oneworld/major_US_airports.htm

Fuel costs:

FlightGlobal, & 16, A. (2024, April 16). Fuel costs - weekly prices 2016-2022. Statista. https://www.statista.com/statistics/526660/aviation-fuel-cost-comparison-across-regions/